In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [2]:
# read a CSV into a dataframe (adjust filename as needed)
events_filename = './data/event.csv'  # or provide a full path

if not os.path.exists(events_filename):
    csv_files = [f for f in os.listdir('.') if f.lower().endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError(f"{events_filename!r} not found and no CSV files in the current directory.")
    filename = csv_files[0]
    print(f"No file named 'data.csv' found. Using first CSV in cwd: {filename}")

events = pd.read_csv(events_filename)
print(f"Loaded {len(events)} rows and {len(events.columns)} columns from {events_filename}")
events.head()

Loaded 107738 rows and 15 columns from ./data/event.csv


/var/folders/_b/sl_t4k5539781f29qf723b080000gn/T/ipykernel_85255/2221732598.py:11: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  events = pd.read_csv(events_filename)


,type,inPlay,isStrike,isBall,pitcherName,catcherName,batterName,pitchCode,pitchType,velocity,coordX,coordY,runners,PAId,eventId
0,PITCH,False,False,True,徐若熙,蔣少宏,陳晨威,B,FF,145,71.11,-91.43,[],1_0,1_0_0
1,PITCH,True,True,False,徐若熙,蔣少宏,陳晨威,H,FF,147,54.86,42.67,"[{'type': 'PA', 'runnerName': '陳晨威', 'isOut': ...",1_0,1_0_1
2,PITCH,False,True,False,徐若熙,蔣少宏,林立,F,FF,145,54.86,22.35,[],1_1,1_1_0
3,PITCH,False,True,False,徐若熙,蔣少宏,林立,SW,SL,135,-2.03,-56.89,[],1_1,1_1_1
4,PITCH,False,False,True,徐若熙,蔣少宏,林立,B,SL,139,-34.54,-144.25,[],1_1,1_1_2


In [3]:
events['coordX'] = events['coordX'] * 0.5 * 0.3937  # convert cm to inches and scale X
events['coordY'] = events['coordY'] * 0.5 * 0.3937  # convert cm to inches and scale Y

In [4]:
# # read a CSV into a dataframe (adjust filename as needed)
# pa_filename = './data/pa.csv'  # or provide a full path

# if not os.path.exists(pa_filename):
#     csv_files = [f for f in os.listdir('.') if f.lower().endswith('.csv')]
#     if not csv_files:
#         raise FileNotFoundError(f"{pa_filename!r} not found and no CSV files in the current directory.")
#     filename = csv_files[0]
#     print(f"No file named 'data.csv' found. Using first CSV in cwd: {filename}")



# pas = pd.read_csv(pa_filename, usecols=['inning', 'batterName', 'pitcherName', 'catcherName', 'paRound', 'pitchCodes', 'result', 'hardness', 'bases'])
# print(f"Loaded {len(pas)} rows and {len(pas.columns)} columns from {pa_filename}")
# pas.head()


In [5]:
# read a CSV into a dataframe (adjust filename as needed)
pa_filename = './data/pa_with_stats.csv'  # or provide a full path

if not os.path.exists(pa_filename):
    csv_files = [f for f in os.listdir('.') if f.lower().endswith('.csv')]
    if not csv_files:
        raise FileNotFoundError(f"{pa_filename!r} not found and no CSV files in the current directory.")
    filename = csv_files[0]
    print(f"No file named 'data.csv' found. Using first CSV in cwd: {filename}")


pas = pd.read_csv(pa_filename, usecols=['batterName', 'batterHand', 'pitcherName', 'pitcherHand', 'bases', 'pitchCodes', 'result', 'hardness'])
print(f"Loaded {len(pas)} rows and {len(pas.columns)} columns from {pa_filename}")
pas.head()


Loaded 27600 rows and 8 columns from ./data/pa_with_stats.csv


,batterName,batterHand,pitcherName,pitcherHand,bases,pitchCodes,result,hardness
0,陳晨威,L,徐若熙,R,0,"['B', 'H']",GO,M
1,林立,R,徐若熙,R,0,"['F', 'SW', 'B', 'B', 'S']",SO,NaN
2,梁家榮,L,徐若熙,R,0,"['S', 'F', 'B', 'H']",2B,H
3,廖健富,L,徐若熙,R,2,"['B', 'H']",1B,H
4,朱育賢,L,徐若熙,R,0,"['S', 'SW', 'H']",FO,M


# Filter out non-pitching events

In [6]:
events = events[events['type'] == 'PITCH']

# Pair PA and Events

In [7]:
# Pair plate-appearances (PA) from `pas`/`pa_filtered` with sequences of pitch `events` in `events_filtered`.
# Algorithm:
#  - Walk through events_filtered in order, accumulate pitches into a current PA.
#  - End a PA when: row['inPlay'] is True OR the row['runners'] contains a dict with type == 'PA' (robust end marker)
#  - Also force-end if batter/pitcher changes between consecutive rows.
#  - Build a pa_events dataframe, assign a sequential pa_seq per (batterName, pitcherName),
#    and then align with pas (or pa_filtered) by the same (batterName, pitcherName, pa_seq).
#
# Note: This cell uses variables already present in the notebook (events_filtered, pas or pa_filtered).
#       It does not re-import modules.

def _row_has_pa_marker(runners, batter_name):
    # runners may be a list of dicts or something else; be defensive
    if not runners:
        return False
    try:
        for r in runners:
            if isinstance(r, dict) and r.get('type') == 'PA':
                # optionally check runnerName matches batter_name if present
                rn = r.get('runnerName')
                if rn is None or rn == batter_name:
                    return True
    except Exception:
        return False
    return False

# ensure chronological order
events_seq = events.sort_index()

# Pitch codes for determining balls and strikes
STRIKE_CODES = {'S', 'SW', 'FT', 'FOUL_BUNT', 'TRY_BUNT'}  # Strikes (including foul bunt w/ 2 strikes)
FOUL_CODES = {'F'}  # Regular fouls
BALL_CODES = {'B'} # Balls

pa_events = []
current = None

for idx, row in events_seq.iterrows():
    b = row['batterName']
    p = row['pitcherName']
    paid = row.get('PAId')
    inplay = row['inPlay']
    code = row.get('pitchCode')

    # start new PA if none
    if current is None:
        current = {
            'PAId': paid,
            # 'inPlay': inplay,
            'batterName': b,
            'pitcherName': p,
            'start_idx': idx,
            'events_idx': [],
            'pitchCodes': [],
            'pitchTypes': [],
            'velocities': [],
            'coordXs': [],
            'coordYs': [],
            'inPlay_flags': [],
            'ball_strike_counts': [],
            'balls': 0,
            'strikes': 0,
            'last_row': None
        }

    # if batter or pitcher changed, finalize previous and start new
    if b != current['batterName'] or p != current['pitcherName']:
        current['end_idx'] = current['events_idx'][-1]
        pa_events.append(current)
        current = {
            'PAId': paid,
            # 'inPlay': inplay,
            'batterName': b,
            'pitcherName': p,
            'start_idx': idx,
            'events_idx': [],
            'pitchCodes': [],
            'pitchTypes': [],
            'velocities': [],
            'coordXs': [],
            'coordYs': [],
            'inPlay_flags': [],
            'ball_strike_counts': [],
            'balls': 0,
            'strikes': 0,
            'last_row': None
        }

    # append to current
    current['events_idx'].append(idx)
    current['pitchCodes'].append(code)
    current['pitchTypes'].append(row.get('pitchType'))
    current['velocities'].append(row.get('velocity'))
    current['coordXs'].append(row.get('coordX'))
    current['coordYs'].append(row.get('coordY'))
    current['inPlay_flags'].append(row.get('inPlay'))
    current['last_row'] = row
    
    # Record ball-strike count *before* this pitch
    current['ball_strike_counts'].append(f"{current['balls']}-{current['strikes']}")

    # Update ball-strike count for the *next* pitch
    if code in BALL_CODES:
        current['balls'] += 1
    elif code in STRIKE_CODES:
        current['strikes'] += 1
    elif code in FOUL_CODES:
        if current['strikes'] < 2:
            current['strikes'] += 1
    
    # end conditions for the PA
    # In-play events, robust PA markers, or reaching 4 balls / 3 strikes
    in_play_codes = {'H', 'Bunt'}
    if row.get('inPlay') or code in in_play_codes or _row_has_pa_marker(row.get('runners'), b) or current['balls'] == 4 or current['strikes'] == 3:
        current['end_idx'] = idx
        pa_events.append(current)
        current = None

# finalize any remaining current PA
if current is not None and current['events_idx']:
    current['end_idx'] = current['events_idx'][-1]
    pa_events.append(current)
    current = None

# build dataframe of pa events
pa_events_df = pd.DataFrame([{
    'PAId': e['PAId'],
    # 'inPlay': e['inPlay'],
    'batterName': e['batterName'],
    'pitcherName': e['pitcherName'],
    # 'start_idx': e['start_idx'],
    # 'end_idx': e['end_idx'],
    # 'events_idx': e['events_idx'],
    'pitchCodes_events': e['pitchCodes'],
    'pitchTypes_events': e['pitchTypes'],
    'velocities_events': e['velocities'],
    'coords_events': list(zip(e['coordXs'], e['coordYs'])),
    'ball_strike_counts': e['ball_strike_counts'],
    # 'inPlay_flags': e['inPlay_flags'],
    'n_pitches': len(e['events_idx'],)
} for e in pa_events])

# assign sequential PA index per (batter, pitcher) to allow pairing with pas
pa_events_df['pa_seq'] = pa_events_df.groupby(['batterName', 'pitcherName']).cumcount() + 1

# prepare pas (use pa_filtered if you prefer pairing only filtered PAs)
pas_seq = pas.copy()
pas_seq = pas_seq.sort_index()  # keep original order
pas_seq['pa_seq'] = pas_seq.groupby(['batterName', 'pitcherName']).cumcount() + 1

# merge on batterName, pitcherName, pa_seq
paired = pd.merge(pas_seq, pa_events_df,
                  on=['batterName', 'pitcherName', 'pa_seq'],
                  how='left',
                  suffixes=('_pa', '_events'))

# show some diagnostics
print(f"Built {len(pa_events_df)} PA event groups from {len(events_seq)} events")
print(f"pas has {len(pas_seq)} rows; merged result has {len(paired)} rows")
display(paired.head(60))
print(paired.keys())

Built 27480 PA event groups from 103620 events
pas has 27600 rows; merged result has 27600 rows


,batterName,batterHand,pitcherName,pitcherHand,bases,pitchCodes,result,hardness,pa_seq,PAId,pitchCodes_events,pitchTypes_events,velocities_events,coords_events,ball_strike_counts,n_pitches
0,陳晨威,L,徐若熙,R,0,"['B', 'H']",GO,M,1,1_0,"[B, H]","[FF, FF]","[145, 147]","[(13.9980035, -17.997995500000002), (10.799191...","[0-0, 1-0]",2.0
1,林立,R,徐若熙,R,0,"['F', 'SW', 'B', 'B', 'S']",SO,NaN,1,1_1,"[F, SW, B, B, S]","[FF, SL, SL, FF, FF]","[145, 135, 139, 149, 150]","[(10.799191, 4.3995975000000005), (-0.39960549...","[0-0, 0-1, 0-2, 1-2, 2-2]",5.0
2,梁家榮,L,徐若熙,R,0,"['S', 'F', 'B', 'H']",2B,H,1,1_2,"[S, F, B, H]","[CU, CU, CH, CH]","[121, 124, 132, 133]","[(-2.3996014999999997, -5.1988085), (-11.99800...","[0-0, 0-1, 0-2, 1-2]",4.0
3,廖健富,L,徐若熙,R,2,"['B', 'H']",1B,H,1,1_3,"[B, H]","[FF, FF]","[146, 144]","[(17.598390000000002, 18.397600999999998), (4....","[0-0, 1-0]",2.0
4,朱育賢,L,徐若熙,R,0,"['S', 'SW', 'H']",FO,M,1,1_4,"[S, SW, H]","[FF, CU, FF]","[149, 124, 148]","[(-2.799207, 0.7992109999999999), (-1.999996, ...","[0-0, 0-1, 0-2]",3.0
5,林承飛,R,徐若熙,R,0,"['SW', 'SW', 'B', 'F', 'F', 'S']",SO,NaN,1,1_5,"[SW, SW, B, F, F, S]","[FF, FF, FF, FF, SL, CU]","[150, 150, 153, 149, 134, 122]","[(-0.7992109999999999, 13.1987925), (3.1988125...","[0-0, 0-1, 0-2, 1-2, 1-2, 1-2]",6.0
6,邱丹,L,徐若熙,R,0,['H'],GO,H,1,1_6,[H],[CH],[133],"[(4.3995975000000005, 7.9980155)]",[0-0],1.0
7,嚴宏鈞,L,徐若熙,R,0,"['B', 'B', 'S', 'H']",GO,S,1,1_7,"[B, B, S, H]","[FF, SL, FF, FF]","[147, 136, 148, 147]","[(10.598404, 17.803114), (-14.8365845, -17.379...","[0-0, 1-0, 2-0, 2-1]",4.0
8,林禹叡,L,徐若熙,R,0,"['B', 'B', 'F', 'S', 'SW']",SO,NaN,1,1_8,"[B, B, F, S, SW]","[SL, FF, FF, FF, CU]","[134, 148, 148, 146, 122]","[(0.8484234999999999, 30.944819999999996), (14...","[0-0, 1-0, 2-0, 2-1, 2-2]",5.0
9,陳晨威,L,徐若熙,R,0,"['B', 'S', 'B', 'F', 'SW']",SO,NaN,2,1_9,"[B, S, B, F, SW]","[CU, FF, FF, FF, CH]","[120, 148, 150, 151, 133]","[(-16.956659, -11.444859), (-1.6948785, -7.629...","[0-0, 1-0, 1-1, 2-1, 2-2]",5.0


Index(['batterName', 'batterHand', 'pitcherName', 'pitcherHand', 'bases',
       'pitchCodes', 'result', 'hardness', 'pa_seq', 'PAId',
       'pitchCodes_events', 'pitchTypes_events', 'velocities_events',
       'coords_events', 'ball_strike_counts', 'n_pitches'],
      dtype='object')


# Pitchers and batters PA>50

In [8]:
# batter_names = paired['batterName'].unique()
# print(f"Found {len(batter_names)} unique batters")

# batter_names_filtered = []  
# for batter in batter_names:
#     pas_batter = paired[paired['batterName'] == batter]
#     if len(pas_batter) < 50:
#         continue
#     batter_names_filtered.append(batter)

# print(f'Filtered to {len(batter_names_filtered)} batters with at least 50 PA each')

In [9]:
# pitcher_names = paired['pitcherName'].unique()
# print(f"Found {len(pitcher_names)} unique pitchers")

# pitcher_names_filtered = []
# for pitcher in pitcher_names:
#     pas_pitcher = paired[paired['pitcherName'] == pitcher]
#     if len(pas_pitcher) < 50:
#         continue
#     pitcher_names_filtered.append(pitcher)

# print(f'Filtered to {len(pitcher_names_filtered)} pitchers with at least 50 PA each')

In [10]:
# paired_filtered = paired[paired['batterName'].isin(batter_names_filtered) & paired['pitcherName'].isin(pitcher_names_filtered) ]
# print(f"Filtered PA count: {len(paired_filtered)}")

# # events_filtered = events[events['batterName'].isin(batter_names_filtered) & events['pitcherName'].isin(pitcher_names_filtered)]
# # print(f"Filtered Event count: {len(events_filtered)}")

# Write csv

In [11]:
out_path = './data/paired_filtered.csv'
os.makedirs(os.path.dirname(out_path) or '.', exist_ok=True)
paired.to_csv(out_path, index=False, encoding='utf-8-sig')
print(f"Wrote {len(paired)} rows x {len(paired.columns)} cols to {out_path!r}")

Wrote 27600 rows x 16 cols to './data/paired_filtered.csv'
